# Imports

In [ ]:
import sys

sys.path.append("..")

from solarv2 import *
from solarv2.mc import *

# Parameters

In [ ]:
# Load options
params.reload_files = True
params.simulate_dead_area = True

# Save options
params.overwrite_metrics = True
params.save_figures = True

# Plotting options
params.flip_x = True
params.individual_plots = np.arange(1, 11, 1)
params.show_figures = False
params.label_font_size = 16
params.tick_font_size = 16
params.title_font_size = 18

# Events to process
params.event_list = None

# Noisy Pixels
params.channel_disable_list = [(7, (1, 2))]  # (chip, channel)


# Transform for real readout
params.detector_x = params.quadrant_size * 8
params.detector_y = params.quadrant_size * 8

# DBSCAN parameters for charge clustering
params.min_samples = 2
params.xy_epsilon = 8  # 8 ideal
params.z_epsilon = 8  # 8 ideal

# RANSAC parameters for line fitting
params.ransac_residual_threshold = params.xy_epsilon - 1  # 6 ideal for charge, 35 ideal for light
params.ransac_max_trials = 1000
params.ransac_min_samples = 2  # 2 ideal for charge, 3 ideal for light

# Force parameters for cylinder
params.force_dh = None
params.force_dr = None

In [ ]:
# File paths
charge_file = "/eos/experiment/dune/solar/montecarlo/singlecube/cosmic_v0/singlecube_cry_hit_0218622392_thrs3800.root"

In [ ]:
params.output_folder = charge_file.split("_")[-1].split(".")[0]
if params.simulate_dead_area:
    params.work_path = "DA"

In [ ]:
recal_params()
translation = get_translation()

# File handing

## Loading

In [ ]:
temp_filename = f"{params.output_folder}/charge_df_{params.output_folder}.bz2"
if not params.reload_files and os.path.isfile(temp_filename):
    charge_df = pd.read_csv(temp_filename, index_col=0)
    if not params.event_list is None:
        charge_df = charge_df.loc[params.event_list.intersection(charge_df.index)]
    # for column in :
    charge_df[charge_df.columns[9:]] = charge_df[charge_df.columns[9:]].applymap(
        lambda x: literal_eval(x) if isinstance(x, str) else x
    )
else:
    charge_df = load_charge(charge_file, events=params.event_list)

del temp_filename

In [ ]:
charge_df = montecarlo.rotate_coordinates(charge_df)

In [ ]:
if params.simulate_dead_area:
    # Cut SiPMs from the anode
    charge_df = montecarlo.cut_sipms(charge_df)
    # Cut dead chips from anode
    charge_df = montecarlo.cut_chips(charge_df)

In [ ]:
# Translate coordinates
if any([t != 0 for t in translation]):
    charge_df = montecarlo.translate_coordinates(charge_df, translation)

In [ ]:
# Cut volume to live anode area
charge_df = montecarlo.cut_volume(charge_df)

## Saving

In [ ]:
os.makedirs(params.output_folder, exist_ok=True)

In [ ]:
# Only save files if all events were considered, i.e. event_list is None
if params.event_list is None:
    charge_df.to_csv(f"{params.output_folder}/charge_df_{params.output_folder}.bz2")

In [ ]:
# Save parameters to JSON just in case
params_to_json(f"{params.output_folder}/reconstruction_parameters_{params.output_folder}.json")

## Verification

In [ ]:
charge_df.columns

In [ ]:
charge_df.count()

### Histograms

#### Charge

In [ ]:
print(f"Hits q in {params.q_unit}")
charge_df["hit_q"].apply(tuple).explode().hist()

In [ ]:
print(f"Hits x in {params.z_unit}")
charge_df["hit_x"].apply(tuple).explode().hist()

In [ ]:
print(f"Hits y in {params.z_unit}")
charge_df["hit_y"].apply(tuple).explode().hist()

In [ ]:
print(f"Hits z in {params.z_unit}")
charge_df["hit_z"].apply(tuple).explode().hist()

# Data fit

## Fake data map

In [ ]:
plot_fake_data(
    [1] if params.simulate_dead_area else [],
    buffer=(params.xy_epsilon - 1),
)
if params.show_figures:
    plt.show()
else:
    plt.close("all")

## Main loop

In [ ]:
# Suppress the UndefinedMetricWarning
warnings.filterwarnings("ignore", category=Warning, module="sklearn")

In [ ]:
metrics = {}

if params.event_list is None:
    index_list = charge_df.index
else:
    index_list = charge_df.index.intersection(params.event_list)

for i, idx in enumerate(tqdm(index_list)):
    charge_values = pd.DataFrame(
        charge_df.loc[
            idx,
            [
                "hit_x",
                "hit_y",
                "hit_z",
                "hit_q",
            ],
        ].to_list(),
        index=["x", "y", "z", "q"],
    ).T

    # charge_values["q"] = charge_values["q"] * charge_gain  # Convert mV to ke

    if len(charge_values) > 2:
        # Create a design matrix
        labels = cluster_hits(
            charge_values[["x", "y", "z"]].to_numpy(),
        )
        # Fit clusters
        metrics[idx] = fit_hit_clusters(
            charge_values[["x", "y", "z"]].to_numpy(),
            charge_values["q"].to_numpy(),
            labels,
            # refit_outliers=False,
        )
        if idx in params.individual_plots:
            _ = event_display(
                event_idx=idx,
                charge_df=charge_values,
                metrics=metrics[idx],
                plot_cyl=False,
            )
            if params.show_figures:
                plt.show()
            else:
                plt.close()

        # metrics[idx][
        #     "Pixel_mask"
        # ] = mask.to_numpy()  # Save masks to original dataframe for reference
        metrics[idx]["Total_charge"] = charge_values["q"].sum()

In [ ]:
# Reset the warning filter (optional)
warnings.filterwarnings("default", category=Warning)

## Metrics

In [ ]:
# Save the metrics to a pickle file
metrics_file = f"{params.output_folder}/metrics_{params.output_folder}.pkl"
if params.overwrite_metrics or not os.path.isfile(metrics_file):
    with open(metrics_file, "wb") as f:
        pickle.dump(metrics, f)

    print(f"Metrics saved to {metrics_file}")

In [ ]:
metrics